In [5]:
# Import necessary libraries
import cv2  # OpenCV for image and video processing
import mediapipe as mp  # Mediapipe for hand tracking and other vision tasks
import time  # To calculate FPS and delays
import math  # Math functions for geometric calculations
import numpy as np  # Numerical operations and array manipulations
from sklearn.linear_model import LinearRegression  # Linear regression model for motion analysis


In [1]:
class HandTrackingDynamic:
    """
    Hand tracking class using MediaPipe to detect hand landmarks, analyze motion, 
    and recognize gestures.

    Attributes:
        mode (bool): Static or dynamic image mode for hand detection.
        maxHands (int): Maximum number of hands to detect.
        detectionCon (float): Minimum confidence value for hand detection.
        trackCon (float): Minimum confidence value for hand tracking.
        handsMp (object): MediaPipe hands solution object.
        hands (object): MediaPipe hands processing object.
        mpDraw (object): MediaPipe drawing utilities.
        tipIds (list): List of landmark IDs corresponding to finger tips.
        arrayX (list): List of x-coordinates of tracked points.
        arrayY (list): List of y-coordinates of tracked points.
    """
    
    def __init__(self, mode=False, maxHands=1, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        # Initialize MediaPipe hands and drawing utilities
        self.handsMp = mp.solutions.hands
        self.hands = self.handsMp.Hands(
            static_image_mode=self.mode,
            max_num_hands=self.maxHands,
            min_detection_confidence=self.detectionCon,
            min_tracking_confidence=self.trackCon
        )
        self.mpDraw = mp.solutions.drawing_utils
        
        # IDs for finger tips (Thumb, Index, Middle, Ring, Pinky)
        self.tipIds = [4, 8, 12, 16, 20]
        
        # Arrays to store coordinates for motion analysis
        self.arrayX = []
        self.arrayY = []
        
        # List of landmarks for detected hands
        self.lmsList = []

    def findFingers(self, frame, draw=True):
        """
        Detects and draws landmarks on hands present in the frame.
        
        Args:
            frame (np.array): The frame from the video feed.
            draw (bool): Whether to draw landmarks on the frame.
        
        Returns:
            frame (np.array): Frame with hand landmarks drawn.
        """
        imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        
        # If hands are detected, draw landmarks
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(frame, handLms, self.handsMp.HAND_CONNECTIONS)
        
        return frame

    def findPosition(self, frame, handNo=0, draw=True):
        """
        Finds the position of landmarks for a specific hand and calculates movement metrics.
        
        Args:
            frame (np.array): The frame from the video feed.
            handNo (int): Index of the hand to track.
            draw (bool): Whether to draw circles on landmarks.
        
        Returns:
            lmsList (list): List of landmark IDs and their coordinates.
            bbox (tuple): Bounding box coordinates for the detected hand.
        """
        xList = []
        yList = []
        bbox = []
        self.lmsList = []

        # Check if hands are detected
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            
            # Loop through each landmark and get coordinates
            for id, lm in enumerate(myHand.landmark):
                h, w, _ = frame.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                xList.append(cx)
                yList.append(cy)
                self.lmsList.append([id, cx, cy])

                # Track wrist movement for linear motion analysis
                if id == 0:
                    self.arrayX.append(cx)
                    self.arrayY.append(cy)
                    
                    # Check if enough points are collected for analysis
                    if len(self.arrayX) == 10:
                        if np.var(self.arrayX) > 30:    
                            sumXY = self.getSumXY()
                            sumXSquared = self.getSumXSquared()
                            slope = self.calculateSlopeEquation(len(self.arrayX), sum(self.arrayX), sum(self.arrayY), sumXY, sumXSquared)
                            yInterceptB = self.calculateYInterceptB(slope, len(self.arrayX))
                            yApproximatedArray = self.getApproximatedArray(slope, yInterceptB)
                            sumOfResidualError = self.getSumOfResidualError(yApproximatedArray)
                            
                            if sumOfResidualError < 50:
                                print("Linear movement detected.")
                            print("Residual errors sum =", sumOfResidualError)
                    
                        self.arrayX = []
                        self.arrayY = []

                # Draw circles on landmarks
                if draw:
                    cv2.circle(frame, (cx, cy), 5, (255, 0, 255), cv2.FILLED)

            # Calculate bounding box for the detected hand
            xmin, xmax = min(xList), max(xList)
            ymin, ymax = min(yList), max(yList)
            bbox = xmin, ymin, xmax, ymax
            
            if draw:
                cv2.rectangle(frame, (xmin - 20, ymin - 20), (xmax + 20, ymax + 20), (0, 255, 0), 2)

        return self.lmsList, bbox

    def calculateYInterceptB(self, slope, numberOfPoints):
        meanX = sum(self.arrayX) / numberOfPoints
        meanY = sum(self.arrayY) / numberOfPoints
        return meanY - (slope * meanX)

    def getSumXY(self):
        return sum([self.arrayX[i] * self.arrayY[i] for i in range(len(self.arrayX))])

    def calculateSlopeEquation(self, numberOfPoints, sumX, sumY, sumXY, sumXSquared):
        numerator = numberOfPoints * sumXY - (sumX * sumY)
        denominator = numberOfPoints * sumXSquared - (sumX ** 2)
        return numerator / denominator

    def getSumXSquared(self):
        return sum([math.pow(x, 2) for x in self.arrayX])

    def getApproximatedArray(self, slope, yInterceptB):
        return [slope * x + yInterceptB for x in self.arrayX]

    def getSumOfResidualError(self, yApproximatedArray):
        return sum([abs(yApproximatedArray[i] - self.arrayY[i]) for i in range(len(self.arrayY))])

In [2]:
def main():
    """
    Main function to start the webcam feed and run hand tracking.
    """
    ctime, ptime = 0, 0
    cap = cv2.VideoCapture(0)
    detector = HandTrackingDynamic()
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    if not cap.isOpened():
        print("Cannot open camera.")
        exit()

    while True:
        ret, frame = cap.read()
        frame = detector.findFingers(frame)
        detector.findPosition(frame)
        
        ctime = time.time()
        fps = 1 / (ctime - ptime)
        ptime = ctime

        cv2.putText(frame, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)
        cv2.imshow('frame', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


In [3]:
if __name__ == "__main__":
    main()

NameError: name 'cv2' is not defined